In [1]:
!pip install imblearn

  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [2]:
train=pd.read_csv('train.csv')
train=train.drop(['ReportTime','Sha256','PeTimestamp','FirstObserved'],axis=1)


In [22]:
print(train.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856674 entries, 0 to 856673
Data columns (total 13 columns):
isPeFile               856674 non-null bool
isValidSignedFile      856674 non-null bool
fileSize               856674 non-null int64
filePrevalence         856674 non-null int64
GeoId                  856674 non-null int64
Extension              782515 non-null object
ImportFunctionCount    709803 non-null float64
ImportModuleCount      709803 non-null float64
PeAppendedSize         709804 non-null float64
PeHeaderChecksum       712833 non-null float64
PeTimestamp            712888 non-null object
FirstObserved          830745 non-null object
Label                  856674 non-null object
dtypes: bool(2), float64(4), int64(3), object(4)
memory usage: 73.5+ MB
None


## Creating dummy variables




In [4]:
cat_vars=['isPeFile','isValidSignedFile','Extension']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(train[var], prefix=var)
    train1=train.join(cat_list)
    train=train1
t_vars=train.columns.values.tolist()
to_keep=[i for i in t_vars if i not in cat_vars]
to_keep
data_final=train[to_keep]
data_final=data_final[data_final['ImportFunctionCount'].notna()]

ValueError: columns overlap but no suffix specified: Index(['isPeFile_False', 'isPeFile_True'], dtype='object')

In [6]:
data_final=data_final.dropna()

# Using Smote to oversample   

In [7]:
# Separate input features and target
y = data_final.Label
X = data_final.drop('Label', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

os = SMOTE(k_neighbors=5, random_state=100) # we can use different values of k


os_data_X,os_data_y=os.fit_sample(X_train, y_train)

In [9]:
print("Number of clean in original data: ",len(y_train[y_train=='clean']))
print("Number of malware data in original data: ",len(y_train[y_train=='malware']))

Number of clean in oversampled data:  510174
Number of malware data in oversampled data:  21991


In [10]:
print("Number of clean in oversampled data: ",len(os_data_y[os_data_y=='clean']))
print("Number of malware data in oversampled data: ",len(os_data_y[os_data_y=='malware']))

Number of clean in oversampled data:  510174
Number of malware data in oversampled data:  510174


# Using Random forest and Tuning the parameters

In [8]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state=100)

In [9]:
import numpy as np
import time
from sklearn.model_selection import GridSearchCV
# GridSearchCV to fine tune the parameters
np.random.seed(100)
start = time.time()

param_dist = {'max_depth':[10,20,50,100],
              'bootstrap':[True, False],           
              'min_samples_split': [5,6,7,10],
              'n_estimators':[100,200,300]
              
             }

cv_rf = GridSearchCV(forest, cv=10, param_grid=param_dist, n_jobs=-1)
cv_rf.fit(os_data_X, os_data_y)
#cv_rf.fit(X_train, y_train)
print('Best parameters using grid search: \n',
       cv_rf.best_params_)
end = time.time()
print('Time taken in grid search: {0: .2f}'.format(end-start))

exception calling callback for <Future at 0x184abd902c8 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
'''
Traceback (most recent call last):
  File "C:\Users\sushmita.biswal\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\sushmita.biswal\AppData\Local\Continuum\anaconda3\lib\multiprocessing\queues.py", line 113, in get
    return _ForkingPickler.loads(res)
MemoryError
'''

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\sushmita.biswal\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\sushmita.biswal\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\parallel.py", line 309, in __call__
    self.paralle

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [11]:
import random
random.seed(999)

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

# check classification scores of random forest
modelRF = forest.set_params(bootstrap=True, max_features=None, max_depth=50,
                            n_estimators=100, min_samples_split=5)
modelRF.fit(os_data_X, os_data_y)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [14]:

y_pred_train = modelRF.predict(X_train)
y_pred = modelRF.predict(X_test)

y_pred_proba_train = modelRF.predict_proba(X_train)[:, 1]
y_pred_proba = modelRF.predict_proba(X_test)[:, 1]




In [20]:
y_pred_proba_train

array([0.03361111, 0.        , 0.002     , ..., 0.        , 0.        ,
       0.        ])

In [21]:
print('Result on train data:\n')
[fpr, tpr, thr] = roc_curve(y_train.map({'clean':0,'malware':1}), y_pred_proba_train)
print(modelRF.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_train, y_pred_train))
print(modelRF.__class__.__name__+" log_loss is %2.3f" % log_loss(y_train, y_pred_proba_train))
print(modelRF.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

Result on train data:

RandomForestClassifier accuracy is 0.999
RandomForestClassifier log_loss is 0.009
RandomForestClassifier auc is 1.000


# Predicting for the test data


In [25]:
### output 
test=pd.read_csv("test.csv")
cat_vars=['isPeFile','isValidSignedFile','Extension']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(train[var], prefix=var)
    test1=test.join(cat_list)
    test=test1
t_vars=test.columns.values.tolist()
to_keep=[i for i in t_vars if i not in cat_vars]
to_keep
data_final=test[to_keep]
data_final=data_final[data_final['ImportFunctionCount'].notna()]
data_final=data_final.dropna()
data_final=data_final.drop(['ReportTime','Sha256','PeTimestamp','FirstObserved'],axis=1)

In [26]:
y_pred = modelRF.predict(data_final)

In [34]:
result=pd.DataFrame(zip(test['Sha256'],y_pred),columns=['Sha256','Label'])
result.to_csv("Sushmita.csv")